In [ ]:
#!/usr/bin/env python
# coding: utf-8

# Tutorial: Modeling The Impact of Structural Lesions -- Part III: Offline Analysis

In this tutorial we will analyze simulated BOLD signals obtained using TVB 1.1-Linux-64 web interface. <br>
<br>
If you want to see the strategies used to model lesions, check the **Tutorial: Modeling The Impact of Structural Lesions  -- Part I: Modeling Lesions**.   <br>
<br>
The parameters used for the neural mass model can be found in the **Tutorial: Modeling The Impact of Structural Lesions  -- Part II: The Brain Network model**.   <br>
The simulated time-series are in the project folder, you can either import this file thorugh the interface or use the individual .h5 files: 

Background<br>
----------<br>
<br>
*Functional connectivity* (FC) in this case refers to the matrix of Pearson correlation coefficients between pair of simulated BOLD time-series. <br>
<br>
If you do not have the BOLD signals, but only the time-series corresponding to the 'neural activity', that's ok. <br>
In TVB you can find several alternatives of the Balloon-Windkessel [1] to compute those offline.<br>


#### Balloon model - BOLD signals

In[5]:

In [ ]:
import tvb.analyzers.fmri_balloon as bold
import tvb.datatypes.time_series as time_series

This function expects a TimeSeries datatype, so it is neccesary to create one instance.

By default this model doesn't perform any operation on the input data. There are some theories that suggest that the neural input is the sum of both the excitatory and inhibitory activity -- if both populations are actually represented in the model. Additionally, <br>
In the case of the balloon model the neural activity is assumed to represent a rate. For that reason, when a model like the one we are using now, described a quantity like the membrane potential, it is possible to apply a transformation to obtain a rate (derivative).<br>
<br>
<br>
A demo script making use of the Balloon model is available at ~/scientific_library/tvb/simulator/demos/bold_fmri_balloon_region.py

#### Simulated (convolution based) BOLD signals <br>
<br>
We'll have a look at the simulated BOLD signals produced by one of tvb monitors: the BOLD monitor. The output is a convolution between a kernel function (HRF) and the neural activity time-series. You can have a look at the **Tutorial: Exploring The BOLD Monitor** where the different HRF kernels are displayed. <br>
<br>
We selected a Gamma kernel. See [1] for more details. The following time-series are the BOLD signals of the control Brain Network Model (no lesion).<br>


In[6]:

In [ ]:
import h5py
import numpy
import matplotlib.pyplot as plt
import glob
matplotlib.rcParams.update({'font.size': 22})

We'll aggregate the 998 regions to 66 regions. To do that we need a mapping. You can find the index vector under <br>
<br>
    ~/simulator/files/connectivity/hagmann_hemisphere_both_subcortical_false_998/<br>


**NOTE:** Change to the appropriate path here:

In[ ]:

In [ ]:
mapping = numpy.loadtxt('/Users/paupau/TVB-trunk/trunk/scientific_library/tvb/simulator/files/connectivity/hagmann_hemisphere_both_subcortical_false_998/region_mapping_raw_998_raw_66.txt')

Create a function to aggregate according to this mapping.

In[ ]:

In [ ]:
def compute_average_time_series(mapping, time_series):
    """
    Aggregate the 998 (nodes) regions into 66 (new_number_f_nodes) ROIs
    Parameters
    ----------
    
    mapping array
            index vector defining the sets of regions in the 998 connectome that will be averaged into one region.
    time_series nd array
            simulated time series of shape time_points x nodes
            
    Returns
    -------
    average_time_series nd array
            'spatially' averaged time series of shape time_points x new_number_of_nodes
            
    
    """
    regions = numpy.unique(mapping)
    average_time_series = numpy.zeros((time_series.shape[0], (len(regions))))
        
    for k in regions:
        ts = time_series[:, mapping == k]
        avg_ts = numpy.mean(ts, axis=1)
        average_time_series[:, k] = avg_ts
    
    return average_time_series

#### Functional Connectivity: Correlation coefficients matrices:<br>
<br>
Unpack the '2013-11-26_17-14_DataTypeGroup_TimeSeries.zip' inside the data/ folder<br>
Unpack the '2013-11-26_18-03_DataTypeGroup_TimeSeries.zip' inside the data/ folder<br>
<br>


#### Results with time-delays<br>
<br>
**NOTE:** Change the root path accordingly - give an absolute path

In[ ]:

In [ ]:
root  = '/Users/paupau/trunk/tvb-handbook/tvbmodl/code/data/2013-11-26_17-14_DataTypeGroup_TimeSeries/' 
flist = glob.glob(root + '*/*.h5')

In[ ]:

30 timepoint per minute - 10 minutes

In [ ]:
FC = numpy.zeros((66, 66, len(flist)))
for f_ix, files in enumerate(flist):
    f   = h5py.File(files)
    FC[:, :, f_ix]  = numpy.corrcoef((compute_average_time_series(mapping, f['data'][60:, 0, :, 0])).T)
    f.close() 

In[ ]:

Delays

In [ ]:
get_ipython().magic('pylab --no-import-all inline')
matplotlib.rcParams.update({'font.size': 22})
fig = plt.figure(2, (20., 20.))
lesion_centres = [57, 86, 138, 162, 194, 247, 308, 323, 360, 439, 472, 555, 584, 636, 662, 692, 746, 810, 821, 856, 938, 971, 'Original']

In [ ]:
cmap = brewer2mpl.get_map('RdBu', 'diverging', 11, reverse=True).mpl_colormap

In [ ]:
for i in range(len(flist)-1):
    
    ax1 = plt.subplot(5, 5, i+1)
    plt.pcolormesh(numpy.flipud(FC[:, :, i]), cmap=cmap, vmax=0.4, vmin=-0.4)
    euclidean_distance = numpy.sqrt((FC[:, :, -1] - FC[:, :, i])**2).sum()
    plt.ylim([0, 66])
    plt.xlim([0, 66])
    ax1.set_xticklabels( () )
    ax1.set_yticklabels( () )
    ax1.set_title('L' + str(lesion_centres[i]) + '. dFC: %d' % euclidean_distance)
    if i ==21:
        if i ==21:
    # [left, bottom, width, height] where all quantities are in fractions of figure width and height. 
            fig.subplots_adjust(right=0.9)
            cbar_ax = fig.add_axes([0.95, 0.12, 0.05, 0.8])
            plt.colorbar(cmap=cmap, cax=cbar_ax)

#### Results without time-delays

In[ ]:

In [ ]:
root  = '/Users/paupau/trunk/tvb-handbook/tvbmodl/code/data/2013-11-26_18-03_DataTypeGroup_TimeSeries/' 
flist = glob.glob(root + '*/*.h5')

In[ ]:

30 timepoint per minute - 10 minutes

In [ ]:
FC_nodelays = numpy.zeros((66, 66, len(flist)))
for f_ix, files in enumerate(flist):
    f   = h5py.File(files)
    FC_nodelays[:, :, f_ix]  = numpy.corrcoef((compute_average_time_series(mapping, f['data'][60:, 0, :, 0])).T)
    f.close() 

In[ ]:

No - Delays

In [ ]:
fig = []
get_ipython().magic('pylab --no-import-all inline')
matplotlib.rcParams.update({'font.size': 22})
fig = plt.figure(3, (20., 20.))
lesion_centres = [57, 86, 138, 162, 194, 247, 308, 323, 360, 439, 472, 555, 584, 636, 662, 692, 746, 810, 821, 856, 938, 971, 'Original']

In [ ]:
for i in range(len(flist)-1):
    ax1 = plt.subplot(5, 5, i+4)
    plt.pcolormesh(numpy.flipud(FC_nodelays[:, :, i]), cmap=cmap, vmax=0.4, vmin=-0.4)
    euclidean_distance = numpy.sqrt((FC_nodelays[:, :, -1] - FC_nodelays[:, :, i])**2).sum()
    plt.ylim([0, 66])
    plt.xlim([0, 66])
    ax1.set_xticklabels( () )
    ax1.set_yticklabels( () )
    ax1.set_title('L' + str(lesion_centres[i]) + '. dFC: %03d' % euclidean_distance)
    if i ==21:
    # [left, bottom, width, height] where all quantities are in fractions of figure width and height. 
      fig.subplots_adjust(left=0.1)
      cbar_ax = fig.add_axes([0., 0.12, 0.05, 0.8])
      plt.colorbar(cmap=cmap, cax=cbar_ax)

In[ ]:

In [ ]:
plt.show()

In[ ]:

In [ ]:
def plot_hist(data, colours=None):
    if colours is None:
        colours = ["#348ABD", "#FF4C4C"]

In [ ]:
    fig, ax = plt.subplots(1, figsize=(20,10))
    rh = abs(data[:11])
    lh = abs(data)[11:]
    xticks = numpy.arange(22)
    even_ticks = xticks[0::2]
    odd_ticks = xticks[1::2]
    r1 = ax.bar(even_ticks+0.5,rh, alpha=0.75, width=1., color=colours[0], lw="3", edgecolor=colours[0], label='RH')
    r2 = ax.bar(odd_ticks+0.5,lh, alpha=0.75, width=1., color=colours[1], lw="3", edgecolor=colours[1], label='LH')
    lesion_centres_labels = ['L057', 'L555', 
                  'L086', 'L584', 
                  'L138', 'L636',
                  'L162', 'L662', 
                  'L194', 'L692', 
                  'L247', 'L746', 
                  'L308', 'L810',
                  'L323', 'L821', 
                  'L360', 'L856', 
                  'L439', 'L938', 
                  'L472', 'L971']
    plt.xlim([0.0, 23])
    plt.ylim([0.0, 3.0])
    ax.set_xticks(xticks+1.0)
    ax.set_xticklabels(lesion_centres_labels, fontsize=16)
    plt.legend()

In[ ]:

In [ ]:
diff_FC = []
diff_FC_nodelays = []
diff_FC_d_nd = []
for i in range(len(flist)):
    diff_FC.append(numpy.sqrt((FC[:, :, i] - FC[:, :, -1])**2).sum())
    diff_FC_nodelays.append(numpy.sqrt((FC_nodelays[:, :, i] - FC_nodelays[:, :, -1])**2).sum())
    diff_FC_d_nd.append(numpy.sqrt((FC[:, :, i] - FC_nodelays[:, :, i])**2).sum())
    
diff_FC = numpy.array(diff_FC[:-1])
diff_FC_nodelays = numpy.array(diff_FC_nodelays[:-1])
diff_FC_d_nd = numpy.array(diff_FC_d_nd[:-1])

In[ ]:

In [ ]:
data = diff_FC_nodelays - diff_FC_nodelays.mean()
plot_hist(data / data.std())

In[ ]:

In [ ]:
data = diff_FC - diff_FC.mean()
plot_hist(data/ data.std())

In[ ]:

In [ ]:
data = diff_FC_d_nd - diff_FC_d_nd.mean()
colours = ["#66C2A5", "#FC8D62", "#8DA0CB", "#E78AC3", "#A6D854", "#ffD92F", "#E5C494"]
plot_hist(data / data.std(), colours[0:2])

In[ ]:

Select the some intersting examples ...

In [ ]:
fig = plt.figure(4, (20., 15.))

Delays

In [ ]:
ax1 = plt.subplot(221)
plt.pcolormesh(numpy.flipud(FC[:, :, -1]), cmap=cmap, vmax=0.4, vmin=-0.4)
plt.ylim([0, 66])
plt.xlim([0, 66])
ax1.set_xticklabels( () )
ax1.set_yticklabels( () )
ax1.set_title('FC - Control - Delays')
plt.colorbar(cmap=cmap)

No delays

In [ ]:
ax2 = plt.subplot(222)
ax2.pcolormesh(numpy.flipud(FC_nodelays[:, :, -1]), cmap=cmap, vmax=0.4, vmin=-0.4)
plt.ylim([0, 66])
plt.xlim([0, 66])
ax2.set_xticklabels( () )
ax2.set_yticklabels( () )
ax2.set_title('FC - Control - No delays')
plt.colorbar(cmap=cmap)

Delays

In [ ]:
ax3 = plt.subplot(223)
plt.pcolormesh(numpy.flipud(FC[:, :, 4]), cmap=cmap, vmax=0.4, vmin=-0.4)
plt.ylim([0, 66])
plt.xlim([0, 66])
ax3.set_xticklabels( () )
ax3.set_yticklabels( () )
ax3.set_title('FC - L194 - Delays')
plt.colorbar(cmap=cmap)

No delays

In [ ]:
ax4 = plt.subplot(224)
ax4.pcolormesh(numpy.flipud(FC_nodelays[:, :, 4]), cmap=cmap, vmax=0.4, vmin=-0.4)
plt.ylim([0, 66])
plt.xlim([0, 66])
ax4.set_xticklabels( () )
ax4.set_yticklabels( () )
ax4.set_title('FC - L194 - No delays')
plt.colorbar(cmap=cmap)

[1] Boynton et al. 2012. Linear systems analysis of the fMRI signal. Neuroimage